In [1]:
import os
import sys

**CURRENT SETTINGS: Embedding and language models, Chunk size, etc.**

In [2]:
#embeddingmodel = "BAAI/bge-m3"
embeddingmodel = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

In [3]:
llmmodel = { "model": "gpt-4o",
                "version":os.environ.get('AZURE_OPENAI_VERSION','2024-08-01-preview'),
                "azure_deployment":"gpt-4o",
                "azure_endpoint":os.environ.get('AZURE_ENDPOINT',''),
                "api_key":os.environ.get('AZURE_OPENAI_KEY','')}

In [4]:
procurement_file_dir = 'C:\\Repos\\CFLA\\KND_2020_07_dokumentacijas_pirsmparbaude'
procurement_file = 'Nolikums (90).pdf'
question_file_path = "C:\\Repos\\vpp-cfla\\questions\\questions.yaml"

In [5]:
chunk_size = 1024
chunk_overlap = 0

**STEP 1: Getting markdown text from the _Project Procurement document_**

In [6]:
from scripts.extractmd import Extractor

In [7]:
ex = Extractor()

In [8]:
procurement_content = ex.convert2markdown(f"{procurement_file_dir}\\{procurement_file}")

Processing C:\Repos\CFLA\KND_2020_07_dokumentacijas_pirsmparbaude\Nolikums (90).pdf...
[========================================]


In [9]:
with open ("tmp2.md", 'w',encoding='utf-8') as ofile:
    print(procurement_content, file = ofile)

**STEP 2: Initializing embedding and llm object**

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [11]:
embedding=HuggingFaceEmbedding(model_name=embeddingmodel,trust_remote_code=True)

In [12]:
print(len(embedding._get_text_embedding("Test string")))

768


In [13]:
from llama_index.llms.azure_openai import AzureOpenAI

In [14]:
llm=AzureOpenAI(azure_deployment=llmmodel["azure_deployment"],
                azure_endpoint=llmmodel["azure_endpoint"],temperature=0.0,
                api_version=llmmodel["version"], api_key=llmmodel["api_key"],
                timeout=120,max_retries=3,top_p=0.0001)

In [15]:
#LLM test
#response = llm.complete("What day is today?")

In [16]:
#response

**STEP 3: Creating FAISS vector index for the procurement document**

In [17]:
from scripts.vectorindex import QnAEngine

In [18]:
qnaengine = QnAEngine(embedding,llm)

In [19]:
ret = await qnaengine.createIndex(procurement_content,"Procurement",chunk_size=chunk_size,chunk_overlap=chunk_overlap)

Generating embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ [ time left: 00:00 ]

100 segments created and vectorized.


In [20]:
if ret == False:
    print("**Failed to create index!**")
    exit
else:
    print("Index is ready.")

Index is ready.


In [21]:
#Retrieval test
similarsegments = qnaengine.getSimilarNodes("Vai iepirkuma priekšmets nav sadalīts daļās?")
print(similarsegments)

[{'text': '12.3.  Līgumam ir pievienots 1.pielikums (Tāme) un tā neatņemama sastāvdaļa ir Uzņēmēja piedāvājums\nKonkursam.\n12.4. Līgums sastādīts divos eksemplāros. viens Līguma eksemplārs atrodas pie\nPasūtītāja, otrs - pie Uzņēmēja. Visiem eksemplāriem ir vienāds juridisks spēks.', 'score': '1.0211231708526611', 'metadata': {'H1': '34', 'FileType': 'Procurement', 'ChunkNr': '92'}}, {'text': '**1.6.** **Informācija par iepirkuma priekšmetu**\n1.6.1. Iepirkuma priekšmets ir remontdarbi Krāslavas pilsētas skolu ēkās saskaņā ar Tehniskajām\nspecifikācijām un līgumu projektiem.\n1.6.2. Iepirkuma priekšmets ir sadalīts divās daļās, pretendents var iesniegt piedāvājumu par vienu\nvai vairākām daļām. Pretendents nedrīkst iesniegt piedāvājuma variantus.\n1.6.2.1. Iepirkuma 1.daļa: “Remontdarbi skolas ēkā Pils ielā 5, Krāslavā”, CPV kods: 45453000-7.\n1.6.2.2. Iepirkuma 2.daļa: “Remontdarbi skolas ēkā N.Rancāna ielā 4, Krāslavā”, CPV kods: 454530007.\n1.6.3. Līgumu izpildes līdzfinansēšana pa

In [22]:
#Generation test
#result = qnaengine.askQuestion("Noskaidro, vai lietotāja apgalvojums ir patiess? Domā soli pa soli, katram solim neveltot vairāk par pieciem vārdiem. Pamato savu atbildi, citējot fragmentu no konteksta. Atbildei jābūt 'jā', 'nē' vai 'kontekstā nav informācijas'. Atbildi ietver kvadrātiekavās '[]'. Ja kontekstā nav informācijas, paskaidro, kāda informācija ir nepieciešama, lai novērtētu apgalvojumu.",
#                              "Pasūtītājs ir nodrošinājis iespēju piegādātājiem iepazīties uz vietas ar iepirkuma papildu dokumentiem, kam tehnisku iemeslu dēļ vai tajos iekļautās informācijas vai komerciālu interešu aizsardzības dēļ nav nodrošināma brīva un tieša elektroniska piekļuve, sākot ar attiecīgā iepirkuma izsludināšanas brīdi",
#                              usecontext=True)
#result

**STEP 4: Processing questions**

In [23]:
import yaml
import re

In [24]:
try:
    with open(question_file_path, 'r', encoding='utf-8') as file:
        question_dictonary = yaml.safe_load(file)

except FileNotFoundError:
    print(f"Error: File '{question_file_path}' not found.")
    exit
except yaml.YAMLError as e:
    print(f"Error parsing YAML file: {e}")
    exit
    

In [25]:
table = []

In [26]:
cod_prompt = "Domā soli pa soli, katram solim neveltot vairāk par pieciem vārdiem. Atbildei jābūt 'jā', 'nē' vai 'kontekstā nav informācijas'."

In [27]:
for singleq in question_dictonary:
    if 'question0' in singleq:
        result0 = qnaengine.askQuestion(cod_prompt,singleq['question'],usecontext=True)
        table.append([singleq['nr'],'',result0.strip()])
        
    result = qnaengine.askQuestion(singleq['prompt'],singleq['question'],usecontext=True)
    
    result = re.sub(r'\n\n+',r'\n',result)
    answer = re.search(r'\[([^\[\]]+)\]',result)

    if answer:
        llmanswer=answer.group(1)
        result = result.replace(f"[{llmanswer}]","").replace(f"Atbilde:","")
        table.append([singleq['nr'],llmanswer,result.strip()])
    else:
        table.append([singleq['nr'],'',result.strip()])        

In [28]:
import pandas as pd
data = pd.DataFrame(table, columns=["Nr", "Atbilde", "Pamatojums"])

In [29]:
from tabulate import tabulate

In [30]:
with open("report.htm", 'w', encoding='utf-8') as ofile:
    print(data.to_html(index=False),file=ofile)
print(tabulate(data, headers='keys', tablefmt='grid', showindex=False))

+------+----------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   Nr | Atbilde                    | Pamatojums                                                                                                                                                                                        |
+======+============================+===================================================================================================================================================================================================+
|    1 |                            | Nepieciešama informācija par iepirkuma plānu un konkrētām izmaiņām, lai novērtētu apgalvojumu.                                                                                                    |
+------+----------------------------+---------------------------